In [1]:
import pandas as pd
from dataAug.tools import dataAugForTestSMOTE, dataAugForTestBorderlineSMOTE
from processTools import processRealData
import torch
from Modelclassify_single_1 import ModelClassify
from targeTools import testThresholdFive, Accuracy

In [2]:
feature_pd = pd.read_csv('/home/kongge/projects/new_protT5/data/T5_578.csv')
labels_pd = pd.read_csv("/home/kongge/projects/new_protT5/data/mutil_label_578.csv")

In [31]:
from classify.targeTools import singleThresholdFive

smote_multiple = {}
for item in range(1, 5, 2):
    G_feature, G_label = dataAugForTestSMOTE(feature_pd, labels_pd, item, 1024)
    R_feature, R_label = processRealData(feature_pd, labels_pd, feature_num=1024)

    train_feature = pd.concat([R_feature, G_feature], axis=0)
    train_label = pd.concat([R_label, G_label], axis=0)

    test_df = pd.read_csv("/home/kongge/projects/new_protT5/data/T5_test_mutil_label_122_finally.csv")

    from torch.utils.data import TensorDataset, DataLoader
    datasetTrain = TensorDataset(torch.tensor(train_feature.values), torch.tensor(train_label.values))
    batch_size = len(train_feature)
    shuffle = True
    dataloaderTrain = DataLoader(datasetTrain, batch_size=batch_size, shuffle=True)

    datasetTest = TensorDataset(torch.tensor(test_df.iloc[:,:1024].values), torch.tensor(test_df.iloc[:,-4:].values))
    # 创建 DataLoader 对象，设置批量大小和是否打乱数据
    batch_size = len(datasetTest)
    dataloaderTest = DataLoader(datasetTest, batch_size=batch_size, shuffle=False)

    model = ModelClassify(feature_num=1024)
    criterion = torch.nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.0001)


    import warnings
    warnings.filterwarnings("ignore")
    GMList = {}
    for epoch in range(300):
        model.train()
        total_loss = 0.0
        for idx, data in enumerate(dataloaderTrain, 0):
            inputs, labels = data
            labels = labels.float()
            inputs = inputs.float()
            out = model(inputs)
            loss = criterion(out, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(dataloaderTrain)
        threshold = 0.5
        labels_cov = (out == out.max(dim=1, keepdim=True)[0]).to(dtype=torch.int32)
        print(f"Epoch [{epoch+1}/{300}], Average Loss: {avg_loss:.4f}, ACC: {Accuracy(labels.int(), labels_cov)}")
        GMScore1 = singleThresholdFive(epoch, model, dataloaderTest)
        GMList[epoch] = GMScore1

    best_key = max(GMList, key=GMList.get)
    best_value = GMList[best_key]
    smote_multiple[item] = best_value

Epoch [1/300], Average Loss: 0.6026, ACC: 0.2678376906318083
epoch:0, GM:0.3169398903846741, OAA:0.29508196721311475, ACC:0.3155737704918033, F1:0.3224044144153595
Epoch [2/300], Average Loss: 0.5962, ACC: 0.26130174291939
epoch:1, GM:0.2469945251941681, OAA:0.22950819672131148, ACC:0.2459016393442623, F1:0.25136610865592957
Epoch [3/300], Average Loss: 0.6000, ACC: 0.26130174291938996
epoch:2, GM:0.23114752769470215, OAA:0.20491803278688525, ACC:0.22950819672131148, F1:0.23770490288734436
Epoch [4/300], Average Loss: 0.5956, ACC: 0.2772331154684096
epoch:3, GM:0.19398906826972961, OAA:0.1721311475409836, ACC:0.19262295081967212, F1:0.19945354759693146
Epoch [5/300], Average Loss: 0.5961, ACC: 0.2798202614379085
epoch:4, GM:0.2393442690372467, OAA:0.21311475409836064, ACC:0.23770491803278687, F1:0.24590162932872772
Epoch [6/300], Average Loss: 0.5862, ACC: 0.27355664488017434
epoch:5, GM:0.25519126653671265, OAA:0.23770491803278687, ACC:0.2540983606557377, F1:0.2595628499984741
Epoch [

In [30]:
smote_multiple

{1: [tensor(0.7099),
  0.7099236641221374,
  0.7099236641221374,
  tensor(0.7099),
  tensor(0.7099),
  tensor(0.7099)],
 3: [tensor(0.7023),
  0.7022900763358778,
  0.7022900763358778,
  tensor(0.7023),
  tensor(0.7023),
  tensor(0.7023)]}